# Walled Maze
This is the 2x2 Walled maze. The wall is between 00 and 01.

In [1]:
import numpy as np
from numpy.random import random_integers as rand
import pyquil
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from scipy import linalg

qvm = QVMConnection()

In [2]:
class Maze_Program:
    def __init__(self,n):
        self.prog = Program()
        self.defineGates()
        self.initialise()
        self.qubits = np.arange(0,17)

        #Change input and steps if necessary
        self.prog += H(5)
        self.prog += H(6)
        for i in range(n):
            self.quantumRandomWalk_3QubitMaze()
        self.prog.measure(5,5).measure(6,6)
            #print(prog)
        
        
    def run_prog(self,trials):
        self.output = qvm.run(self.prog, [5,6], trials = trials)
        self.output = np.array(self.output)
        summed = self.output[:,1] * 2. + self.output[:,0]
        #print(summed)
        unique, counts = np.unique(summed,return_counts=True)
        self.results = dict(zip(unique, counts))
        print(self.results)
    

    def createToffoli(self):
        identity = np.identity(8)
        identity[[6],[6]] = 0
        identity[[6],[7]] = 1
        identity[[7],[7]] = 0
        identity[[7],[6]] = 1
        return identity

    def inverseToffoli(self):
        identity = np.identity(8)
        identity[[0],[0]] = 0
        identity[[0],[1]] = 1
        identity[[1],[1]] = 0
        identity[[1],[0]] = 1
        return identity

    def controlSWAPdef(self):
        identity = np.identity(8)
        identity[[5],[5]] = 0
        identity[[5],[6]] = 1
        identity[[6],[6]] = 0
        identity[[6],[5]] = 1
        return identity

    def controlHadamard(self):
        identity = np.identity(4)
        identity[[2],[2]] = 1/np.sqrt(2)
        identity[[2],[3]] = 1/np.sqrt(2)
        identity[[3],[2]] = 1/np.sqrt(2)
        identity[[3],[3]] = -1/np.sqrt(2)
        return identity

    

    #in this  random walk, the graph registers x and y take up 2 qubits, similarly for the ancillas and \psi
    def quantumRandomWalk_3QubitMaze(self):
        self.prog.inst(('Toffoli', 0,4,5))
        self.prog.inst(('Toffoli', 1,4,6))
        self.prog.inst(('invToffoli', 0,4,5))
        self.prog.inst(('invToffoli', 1,4,6))

        self.prog.inst(('Toffoli', 2,4,7))
        self.prog.inst(('Toffoli', 3,4,8))
        self.prog.inst(('invToffoli', 2,4,7))
        self.prog.inst(('invToffoli', 3,4,8))

        self.prog.inst(('Toffoli', 9,11,13))
        self.prog.inst(('Toffoli', 10,12,14))

        self.prog.inst(('Toffoli', 4,13,15))
        self.prog.inst(('Toffoli', 4,14,16))

        self.prog.inst(('controlSWAP', 15,7,5))
        self.prog.inst(('controlSWAP', 16,8,6))

        self.prog.inst(('Toffoli', 4,13,15))
        self.prog.inst(('Toffoli', 4,14,16))

        self.prog.inst(('Toffoli', 9,11,13))
        self.prog.inst(('Toffoli', 10,12,14))

        self.prog.inst(('Toffoli', 2,4,7))
        self.prog.inst(('Toffoli', 3,4,8))
        self.prog.inst(('invToffoli', 2,4,7))
        self.prog.inst(('invToffoli', 3,4,8))

        self.prog.inst(('Toffoli', 0,4,5))
        self.prog.inst(('Toffoli', 1,4,6))
        self.prog.inst(('invToffoli', 0,4,5))
        self.prog.inst(('invToffoli', 1,4,6))

        self.prog.inst(('controlH', 5,7))
        self.prog.inst(('controlH', 6,8))
    

    def defineGates(self):
        controlH = self.controlHadamard()
        self.prog.defgate('controlH', controlH)

        controlSWAPgate = self.controlSWAPdef() 
        self.prog.defgate('controlSWAP', controlSWAPgate)

        invToffoli = self.inverseToffoli()
        self.prog.defgate('invToffoli', invToffoli)

        Toffoli = self.createToffoli()
        self.prog.defgate('Toffoli', Toffoli)


    def initialise(self):
        data = np.loadtxt("data")
        def create_wrong_decimals(list_in_binary):
            out_list = (list_in_binary[:,0] * 2**4 + list_in_binary[:,1] * 2**3 + list_in_binary[:,2] * 2**2 + 
                   list_in_binary[:,3] * 2**1 + list_in_binary[:,4])
            return out_list

        list_in_binary = np.array([[0,0,1,0,0],[ 0,1,1,1,0],[ 1,0,0,0,0],[ 1,0,1,1,0], [1,1,0,1,0], [1,1,1,0,0]])
        wrong_decimals = create_wrong_decimals(list_in_binary) /2

        self.prog += H(0)
        self.prog += H(1)
        self.prog += H(2)
        self.prog += H(3)
        self.prog += I(4)

        dict = {}
        for i in range(2**4):
            if i in wrong_decimals:
                dict[i] = np.array([[0.,1.],[1.,0.]])
            else:
                dict[i] = np.array([[1.,0],[0.,1.]])
        init_gate = linalg.block_diag(dict[0], dict[1], dict[2], dict[3], dict[4], dict[5], dict[6], dict[7], dict[8], dict[9],
                     dict[10],
                     dict[11], dict[12], dict[13],dict[14],dict[15])
        self.prog.defgate("INIT", init_gate) 
        self.prog.inst(("INIT",0,1,2,3,4))

        #results = qvm.run(prog,[0,1,2],1)
        #print(results)

In [3]:
program = Maze_Program(1) #how many steps in quantum walk
program.run_prog(100) #how many trials for the average

UnknownApiError: {'message': 'Endpoint request timed out'}

The server has failed to return a proper response. Please describe the problem
and copy the above message into a GitHub issue at:
    https://github.com/rigetticomputing/pyquil/issues

## Doing State Tomography

In [ ]:
from grove.tomography.state_tomography import do_state_tomography
from grove.tomography.utils import notebook_mode
from pyquil.api import QPUConnection, get_devices, Job
from mock import MagicMock
import json
from __future__ import print_function
import matplotlib.pyplot as plt

qubits = [5,6]
qvm = QVMConnection()
NUM_SAMPLES=2000
# QPU
online_devices = [d for d in get_devices() if d.is_online()]
if online_devices:
    d = online_devices[0]
    qpu = QPUConnection(d.name)
    print("Found online device {}, making QPUConnection".format(d.name))
else:
    qpu = QVMConnection()
    print("Could not find online device, defaulting to QVM")

In [ ]:
acorn = get_devices(as_dict=True)['19Q-Acorn']
qpu = QVMConnection(acorn)

In [ ]:
%%time
#print("Running state tomography on the QPU...")
#state_tomography_qpu, _, _ = do_state_tomography(prog, NUM_SAMPLES, qpu, qubits)
#print("State tomography completed.")
print("Running state tomography on the QVM for reference...")
state_tomography_qvm, _, _ = do_state_tomography(prog, NUM_SAMPLES, qvm, qubits)
print("State tomography completed.")

In [ ]:

#state_fidelity = state_tomography_qpu.fidelity(state_tomography_qvm.rho_est)

#qpu_plot = state_tomography_qpu.plot();
#qpu_plot.text(0.35, 0.9, r'$Fidelity={:1.1f}\%$'.format(state_fidelity*100), size=20)

state_tomography_qvm.plot();